In [2]:
""" numeric project, includes random number generation, tests and area estimation """
from collections import Counter
from scipy.stats import chisquare

# 1.1a Random Number Generation

# implementing a simple pseude-random number generator
# Linear Congruential Generator (LCG)
class LCG:
    def __init__(self, seed=1):
        self.m = 2**31  
        self.a = 22695477
        self.c = 1
        self.seed = seed
        self.x = seed

    def random(self):
        self.x = (self.a * self.x + self.c) % self.m
        return self.x / self.m
    

In [37]:
# Generate 100000 random numbers
lcg = LCG()
random_numbers = [lcg.random() for _ in range(100000)] 

peak memory: 250.47 MiB, increment: 0.00 MiB


Timer unit: 1e-07 s

In [5]:
# 1.1b Cryptographicly Secure Random Number Generator
import secrets

def generate_random_numbers(n):
    """Generates a list of n random numbers using a CSPRNG."""
    return [secrets.SystemRandom().random() for _ in range(n)]

def csprng_random():
    return secrets.SystemRandom().random()

In [38]:
# Generate 100,000 random numbers
random_numbers_C = generate_random_numbers(100000)

peak memory: 250.49 MiB, increment: 0.00 MiB


Timer unit: 1e-07 s

In [39]:
import random

# Generate 100,000 random numbers
random_numbers_module = [random.random() for _ in range(100000)] 

peak memory: 251.51 MiB, increment: 0.00 MiB


Timer unit: 1e-07 s

In [40]:
# 1.1c Diehard Tests
import math
from collections import Counter
from scipy.special import gammaincc

def diehard_tests(random_numbers):
    """Performs 5 selected Diehard tests on a sequence of random numbers."""

    def test_frequency(bits):
        """Frequency Test: Checks if 0s and 1s are equally likely."""
        ones = bits.count('1')
        zeros = bits.count('0')
        return abs(ones - zeros) <= 3 * math.sqrt(len(bits))

    def test_runs(bits):
        """Runs Test: Checks for patterns in consecutive bits."""
        runs = 1
        for i in range(1, len(bits)):
            if bits[i] != bits[i - 1]:
                runs += 1
        expected_runs = 2 * len(bits) / 3
        return abs(runs - expected_runs) <= 1.96 * math.sqrt(len(bits))

    def test_longest_run(bits):
        """Longest Run of Ones Test: Checks for long sequences of 1s."""
        longest_run = 0
        current_run = 0
        for bit in bits:
            if bit == '1':
                current_run += 1
            else:
                longest_run = max(longest_run, current_run)
                current_run = 0
        return longest_run <= 36  # For a sample size of 1 million bits

    def test_serial(bits):
        """Serial Test: Checks for patterns in pairs of consecutive bits."""
        pairs = [bits[i:i + 2] for i in range(0, len(bits) - 1, 2)]
        counts = {'00': 0, '01': 0, '10': 0, '11': 0}
        for pair in pairs:
            counts[pair] += 1
        expected = len(pairs) / 4
        chi_squared = sum([(counts[key] - expected) ** 2 / expected for key in counts])
        return chi_squared <= 9.488  # Chi-squared critical value for df=3 and alpha=0.05

    def test_autocorrelation(data, lag=1):
        """Autocorrelation Test: Checks for correlation between values at different lags."""
        n = len(data)
        mean = sum(data) / n
        variance = sum([(x - mean) ** 2 for x in data]) / (n - 1)

        if variance == 0:  # Handle cases where variance is zero
            return True

        autocorr = sum([(data[i] - mean) * (data[i + lag] - mean) for i in range(n - lag)]) / ((n - lag) * variance)
        return abs(autocorr) < 2 / math.sqrt(n)  # Approximate threshold for significance

    def test_approximate_entropy(bits, m=10):
        """
        Approximate Entropy Test: Checks the randomness of blocks of consecutive bits.
        """
        n = len(bits)
        if n < m:
            return 0.0, False

        def get_counts(seq):
            return dict(Counter([seq[i:i + m] for i in range(len(seq) - m + 1)]))

        C_m = [val / float(n - m + 1) for val in get_counts(bits).values()]
        C_m1 = [val / float(n - m) for val in
                 get_counts(bits[:n - 1]).values()]

        phi_m = sum([c * math.log(c) for c in C_m if c > 0])
        phi_m1 = sum([c * math.log(c) for c in C_m1 if c > 0])

        ApEn_m = phi_m1 - phi_m
        chi_sq = 2 * (n - m + 1) * (math.log(2) - ApEn_m)
        p_value = 1 - gammaincc(2 ** (m - 1), chi_sq / 2.0)

        return p_value, p_value >= 0.01

    # Convert random numbers to bits
    bit_string = ''.join(['1' if x >= 0.5 else '0' for x in random_numbers])

    # Run the selected Diehard tests
    print("Diehard Test Results:")
    print(f"Frequency Test: {'Passed' if test_frequency(bit_string) else 'Failed'}")
    print(f"Runs Test: {'Passed' if test_runs(bit_string) else 'Failed'}")
    print(f"Longest Run of Ones Test: {'Passed' if test_longest_run(bit_string) else 'Failed'}")
    print(f"Serial Test: {'Passed' if test_serial(bit_string) else 'Failed'}")
    print(f"Autocorrelation Test: {'Passed' if test_autocorrelation(random_numbers) else 'Failed'}")
    p_value, passed = test_approximate_entropy(bit_string) # Pass the bit string
    print(f"Approximate Entropy Test: p-value = {p_value:.4f}, Passed = {passed}")

diehard_tests(random_numbers)


Diehard Test Results:
Frequency Test: Passed
Runs Test: Failed
Longest Run of Ones Test: Passed
Serial Test: Passed
Autocorrelation Test: Passed
Approximate Entropy Test: p-value = 1.0000, Passed = True


In [41]:
# 1.1d Statistical Tests
import math
from collections import Counter
from scipy.special import gammaincc
from scipy.stats import chisquare, chi2, expon  # Import chisquare for goodness-of-fit test
import numpy as np


def next_bit_test(bits, k=10):
    """
    Next-Bit Test: Tries to predict the next bit in the sequence based on the preceding k bits. 
    A good RNG should make this prediction difficult. 

    Args:
        bits: A string of '0's and '1's representing the bit sequence.
        k: The number of preceding bits to use for prediction.

    Returns:
        (p-value, bool): The p-value of the test and True if the sequence passes (p-value >= 0.01), False otherwise.
    """
    n = len(bits)
    if n <= k:
        return 0.0, False  # Not enough bits for the test

    # Create a dictionary to store the counts of 0s and 1s following each k-bit pattern
    counts = {}
    for i in range(n - k):
        pattern = bits[i:i + k]
        next_bit = bits[i + k]
        if pattern not in counts:
            counts[pattern] = {'0': 0, '1': 0}
        counts[pattern][next_bit] += 1

    # Calculate the chi-squared statistic
    chi_sq = 0
    for pattern, pattern_counts in counts.items():
        total_count = pattern_counts['0'] + pattern_counts['1']
        if total_count > 0:  # Avoid division by zero
            expected_count = total_count / 2  # Expect 50/50 distribution if random
            chi_sq += ((pattern_counts['0'] - expected_count)**2 / expected_count) + \
                    ((pattern_counts['1'] - expected_count)**2 / expected_count)

    # Calculate the p-value using the chi-squared distribution
    degrees_of_freedom = len(counts) - 1
    p_value = gammaincc(degrees_of_freedom / 2, chi_sq / 2)

    return p_value, p_value >= 0.01

def test_universal_statistical(random_numbers):
    """Universal Statistical Test: 
       - Uses chi-squared goodness-of-fit test for more accurate assessment.
       - Calculates degrees of freedom based on binning.
    """
    n = len(random_numbers)
    num_bins = int(math.sqrt(n))  # Rule of thumb for number of bins
    observed, _ = np.histogram(random_numbers, bins=num_bins) # Use numpy for histogram

    # Assuming uniform distribution as the expected
    expected = np.ones_like(observed) * (n / num_bins)
    chi_sq, p_value = chisquare(observed, expected)  

    degrees_of_freedom = num_bins - 1
    critical_value = chi2.ppf(0.99, df=degrees_of_freedom) # Critical value for 0.01 sig
    
    return chi_sq, chi_sq < critical_value, p_value # Returns statistic, result and p-value

def test_covariance(random_numbers):
    """Covariance Test: 
       - Uses correlation and a t-test for a more statistically sound approach.
    """
    from scipy.stats import pearsonr, t
    n = len(random_numbers)
    if n <= 2: # Not enought data
        return 0, False, 1.0
    
    # Use Pearson's correlation 
    correlation, p_value = pearsonr(random_numbers[:-1], random_numbers[1:])
    
    # Calculate the t-statistic
    t_statistic = correlation * math.sqrt((n - 2) / (1 - correlation**2))

    # Degrees of freedom
    degrees_of_freedom = n - 2

    # Two-tailed p-value (testing for non-zero correlation)
    p_value = 2 * t.sf(abs(t_statistic), df=degrees_of_freedom)

    return correlation, abs(correlation) < 2 / math.sqrt(n), p_value

def test_birthday_spacings(random_numbers):
    """Birthday Spacings Test: 
       - Uses a chi-squared goodness-of-fit test (more accurate).
       - Handles zero expected frequencies appropriately for exponential distribution. 
    """
    n = len(random_numbers)
    if n <= 1:
        return 0, False, 1.0

    sorted_numbers = sorted(random_numbers)
    spacings = [sorted_numbers[i+1] - sorted_numbers[i] for i in range(n-1)]

    # 1. Estimate the rate parameter (lambda) of the exponential distribution
    rate_lambda = 1 / np.mean(spacings)

    # 2. Binning and calculating expected frequencies 
    num_bins = int(math.sqrt(n - 1)) 
    observed, bin_edges = np.histogram(spacings, bins=num_bins, range=(0, 1)) 

    # Using the CDF (cumulative distribution function) to get expected probabilities
    expected_probabilities = expon.cdf(bin_edges[1:], scale=1/rate_lambda) - expon.cdf(bin_edges[:-1], scale=1/rate_lambda)
    expected = expected_probabilities * len(spacings)

    # --- Handling zero expected frequencies for the exponential distribution ---
    # If any expected frequencies are still too low, you might need to adjust
    # the binning strategy or consider if the exponential distribution is 
    # truly a good fit for your data.

    # --- Perform the chi-squared test ---
    chi_squared, p_value = chisquare(observed, expected, ddof=1) 
    degrees_of_freedom = num_bins - 1 - 1 # Subtract 1 for rate estimation 
    critical_value = chi2.ppf(0.99, df=degrees_of_freedom)

    return chi_squared, chi_squared < critical_value, p_value


def test_overlapping_permutations(random_numbers, k=5): 
    """Overlapping Permutations Test: 
       - Uses a chi-squared goodness-of-fit test (more statistically sound).
       - Parameter 'k' controls permutation length (default to 5, can be adjusted).
    """
    n = len(random_numbers)
    if n <= k:
        return 0, False, 1.0

    observed = Counter([tuple(random_numbers[i:i+k]) for i in range(n - k + 1)])

    # Expected frequency for each permutation (assuming uniformity)
    expected_count = (n - k + 1) / math.factorial(k) 

    # Chi-squared calculation 
    chi_sq = sum(((count - expected_count)**2) / expected_count for count in observed.values())
    degrees_of_freedom = len(observed) - 1

    p_value = gammaincc(degrees_of_freedom / 2, chi_sq / 2)
    return chi_sq, p_value >= 0.01, p_value # Returns statistic, result and p-value

def test_binary_matrix_rank(random_numbers, matrix_size=32):
    """Binary Matrix Rank Test:
       - Generates and tests multiple matrices (more robust).
       - Handles cases where the matrix size doesn't divide evenly.
    """
    n = len(random_numbers)
    num_matrices = n // matrix_size  # Number of full matrices we can create
    ranks = []
    for i in range(num_matrices):
        start = i * matrix_size
        end = start + matrix_size
        matrix = [int(x >= 0.5) for x in random_numbers[start:end]]
        rank = len(set(tuple(matrix[i:i+matrix_size]) for i in range(0, matrix_size, matrix_size)))
        ranks.append(rank)
    # Analyze the distribution of ranks. 
    # (You'd need to compare the rank distribution to the expected distribution 
    #  for random matrices, which is more involved)
    return ranks, True, 1.0 # Placeholder: needs rank distribution comparison


def statistical_tests(random_numbers):
    # Existing Diehard tests...
    bit_string = ''.join(['1' if x >= 0.5 else '0' for x in random_numbers])
    p_value, passed = next_bit_test(bit_string)

    print("Additional Statistical Tests:")
    print(f"Universal Statistical Test: {'Passed' if test_universal_statistical(random_numbers) else 'Failed'}")
    print(f"Covariance Test: {'Passed' if test_covariance(random_numbers) else 'Failed'}")
    print(f"Birthday Spacings Test: {'Passed' if test_birthday_spacings(random_numbers) else 'Failed'}")
    print(f"Overlapping Permutations Test: {'Passed' if test_overlapping_permutations(random_numbers) else 'Failed'}")
    print(f"Binary Matrix Rank Test: {'Passed' if test_binary_matrix_rank(random_numbers) else 'Failed'}")
    print(f"Next-Bit Test: p-value = {p_value:.4f}, Passed = {passed}")

statistical_tests(random_numbers)


Additional Statistical Tests:
Universal Statistical Test: Passed
Covariance Test: Passed
Birthday Spacings Test: Passed
Overlapping Permutations Test: Passed
Binary Matrix Rank Test: Passed
Next-Bit Test: p-value = 0.7284, Passed = True


In [42]:
# 1.1c Diehard Tests for CSPRNG
import math
from collections import Counter
from scipy.special import gammaincc

def diehard_tests(random_numbers_C):
    """Performs 5 selected Diehard tests on a sequence of random numbers."""

    def test_frequency(bits):
        """Frequency Test: Checks if 0s and 1s are equally likely."""
        ones = bits.count('1')
        zeros = bits.count('0')
        return abs(ones - zeros) <= 3 * math.sqrt(len(bits))

    def test_runs(bits):
        """Runs Test: Checks for patterns in consecutive bits."""
        runs = 1
        for i in range(1, len(bits)):
            if bits[i] != bits[i - 1]:
                runs += 1
        expected_runs = 2 * len(bits) / 3
        return abs(runs - expected_runs) <= 1.96 * math.sqrt(len(bits))

    def test_longest_run(bits):
        """Longest Run of Ones Test: Checks for long sequences of 1s."""
        longest_run = 0
        current_run = 0
        for bit in bits:
            if bit == '1':
                current_run += 1
            else:
                longest_run = max(longest_run, current_run)
                current_run = 0
        return longest_run <= 36  # For a sample size of 1 million bits

    def test_serial(bits):
        """Serial Test: Checks for patterns in pairs of consecutive bits."""
        pairs = [bits[i:i + 2] for i in range(0, len(bits) - 1, 2)]
        counts = {'00': 0, '01': 0, '10': 0, '11': 0}
        for pair in pairs:
            counts[pair] += 1
        expected = len(pairs) / 4
        chi_squared = sum([(counts[key] - expected) ** 2 / expected for key in counts])
        return chi_squared <= 9.488  # Chi-squared critical value for df=3 and alpha=0.05

    def test_autocorrelation(data, lag=1):
        """Autocorrelation Test: Checks for correlation between values at different lags."""
        n = len(data)
        mean = sum(data) / n
        variance = sum([(x - mean) ** 2 for x in data]) / (n - 1)

        if variance == 0:  # Handle cases where variance is zero
            return True

        autocorr = sum([(data[i] - mean) * (data[i + lag] - mean) for i in range(n - lag)]) / ((n - lag) * variance)
        return abs(autocorr) < 2 / math.sqrt(n)  # Approximate threshold for significance

    def test_approximate_entropy(bits, m=10):
        """
        Approximate Entropy Test: Checks the randomness of blocks of consecutive bits.
        """
        n = len(bits)
        if n < m:
            return 0.0, False

        def get_counts(seq):
            return dict(Counter([seq[i:i + m] for i in range(len(seq) - m + 1)]))

        C_m = [val / float(n - m + 1) for val in get_counts(bits).values()]
        C_m1 = [val / float(n - m) for val in
                 get_counts(bits[:n - 1]).values()]

        phi_m = sum([c * math.log(c) for c in C_m if c > 0])
        phi_m1 = sum([c * math.log(c) for c in C_m1 if c > 0])

        ApEn_m = phi_m1 - phi_m
        chi_sq = 2 * (n - m + 1) * (math.log(2) - ApEn_m)
        p_value = gammaincc(2 ** (m - 1), chi_sq / 2.0)

        return p_value, p_value >= 0.01

    # Convert random numbers to bits
    bit_string = ''.join(['1' if x >= 0.5 else '0' for x in random_numbers_C])

    # Run the selected Diehard tests
    print("Diehard Test Results:")
    print(f"Frequency Test: {'Passed' if test_frequency(bit_string) else 'Failed'}")
    print(f"Runs Test: {'Passed' if test_runs(bit_string) else 'Failed'}")
    print(f"Longest Run of Ones Test: {'Passed' if test_longest_run(bit_string) else 'Failed'}")
    print(f"Serial Test: {'Passed' if test_serial(bit_string) else 'Failed'}")
    print(f"Autocorrelation Test: {'Passed' if test_autocorrelation(random_numbers_C) else 'Failed'}")
    p_value, passed = test_approximate_entropy(bit_string) # Pass the bit string
    print(f"Approximate Entropy Test: p-value = {p_value:.4f}, Passed = {passed}")

diehard_tests(random_numbers_C)


Diehard Test Results:
Frequency Test: Passed
Runs Test: Failed
Longest Run of Ones Test: Passed
Serial Test: Passed
Autocorrelation Test: Passed
Approximate Entropy Test: p-value = 0.0000, Passed = False


In [43]:
import random
import math
from collections import Counter
from scipy.special import gammaincc
from scipy.stats import chisquare, chi2, expon  # Import chisquare for goodness-of-fit test
import numpy as np

# 1.1d Statistical Tests for CSPRNG

def next_bit_test(bits, k=10):
    """
    Next-Bit Test: Tries to predict the next bit in the sequence based on the preceding k bits. 
    A good RNG should make this prediction difficult. 

    Args:
        bits: A string of '0's and '1's representing the bit sequence.
        k: The number of preceding bits to use for prediction.

    Returns:
        (p-value, bool): The p-value of the test and True if the sequence passes (p-value >= 0.01), False otherwise.
    """
    n = len(bits)
    if n <= k:
        return 0.0, False  # Not enough bits for the test

    # Create a dictionary to store the counts of 0s and 1s following each k-bit pattern
    counts = {}
    for i in range(n - k):
        pattern = bits[i:i + k]
        next_bit = bits[i + k]
        if pattern not in counts:
            counts[pattern] = {'0': 0, '1': 0}
        counts[pattern][next_bit] += 1

    # Calculate the chi-squared statistic
    chi_sq = 0
    for pattern, pattern_counts in counts.items():
        total_count = pattern_counts['0'] + pattern_counts['1']
        if total_count > 0:  # Avoid division by zero
            expected_count = total_count / 2  # Expect 50/50 distribution if random
            chi_sq += ((pattern_counts['0'] - expected_count)**2 / expected_count) + \
                    ((pattern_counts['1'] - expected_count)**2 / expected_count)

    # Calculate the p-value using the chi-squared distribution
    degrees_of_freedom = len(counts) - 1
    p_value = gammaincc(degrees_of_freedom / 2, chi_sq / 2)

    return p_value, p_value >= 0.01

def test_universal_statistical(random_numbers_C_C):
    """Universal Statistical Test: 
       - Uses chi-squared goodness-of-fit test for more accurate assessment.
       - Calculates degrees of freedom based on binning.
    """
    n = len(random_numbers_C)
    num_bins = int(math.sqrt(n))  # Rule of thumb for number of bins
    observed, _ = np.histogram(random_numbers_C, bins=num_bins) # Use numpy for histogram

    # Assuming uniform distribution as the expected
    expected = np.ones_like(observed) * (n / num_bins)
    chi_sq, p_value = chisquare(observed, expected)  

    degrees_of_freedom = num_bins - 1
    critical_value = chi2.ppf(0.99, df=degrees_of_freedom) # Critical value for 0.01 sig
    
    return chi_sq, chi_sq < critical_value, p_value # Returns statistic, result and p-value

def test_covariance(random_numbers_C):
    """Covariance Test: 
       - Uses correlation and a t-test for a more statistically sound approach.
    """
    from scipy.stats import pearsonr, t
    n = len(random_numbers_C)
    if n <= 2: # Not enought data
        return 0, False, 1.0
    
    # Use Pearson's correlation 
    correlation, p_value = pearsonr(random_numbers_C[:-1], random_numbers_C[1:])
    
    # Calculate the t-statistic
    t_statistic = correlation * math.sqrt((n - 2) / (1 - correlation**2))

    # Degrees of freedom
    degrees_of_freedom = n - 2

    # Two-tailed p-value (testing for non-zero correlation)
    p_value = 2 * t.sf(abs(t_statistic), df=degrees_of_freedom)

    return correlation, abs(correlation) < 2 / math.sqrt(n), p_value

def test_birthday_spacings(random_numbers_C):
    """Birthday Spacings Test: 
       - Uses a chi-squared goodness-of-fit test (more accurate).
       - Handles zero expected frequencies appropriately for exponential distribution. 
    """
    n = len(random_numbers_C)
    if n <= 1:
        return 0, False, 1.0

    sorted_numbers = sorted(random_numbers_C)
    spacings = [sorted_numbers[i+1] - sorted_numbers[i] for i in range(n-1)]

    # 1. Estimate the rate parameter (lambda) of the exponential distribution
    rate_lambda = 1 / np.mean(spacings)

    # 2. Binning and calculating expected frequencies 
    num_bins = int(math.sqrt(n - 1)) 
    observed, bin_edges = np.histogram(spacings, bins=num_bins, range=(0, 1)) 

    # Using the CDF (cumulative distribution function) to get expected probabilities
    expected_probabilities = expon.cdf(bin_edges[1:], scale=1/rate_lambda) - expon.cdf(bin_edges[:-1], scale=1/rate_lambda)
    expected = expected_probabilities * len(spacings)

    # --- Handling zero expected frequencies for the exponential distribution ---
    # If any expected frequencies are still too low, you might need to adjust
    # the binning strategy or consider if the exponential distribution is 
    # truly a good fit for your data.

    # --- Perform the chi-squared test ---
    chi_squared, p_value = chisquare(observed, expected, ddof=1) 
    degrees_of_freedom = num_bins - 1 - 1 # Subtract 1 for rate estimation 
    critical_value = chi2.ppf(0.99, df=degrees_of_freedom)

    return chi_squared, chi_squared < critical_value, p_value


def test_overlapping_permutations(random_numbers_C, k=5): 
    """Overlapping Permutations Test: 
       - Uses a chi-squared goodness-of-fit test (more statistically sound).
       - Parameter 'k' controls permutation length (default to 5, can be adjusted).
    """
    n = len(random_numbers_C)
    if n <= k:
        return 0, False, 1.0

    observed = Counter([tuple(random_numbers_C[i:i+k]) for i in range(n - k + 1)])

    # Expected frequency for each permutation (assuming uniformity)
    expected_count = (n - k + 1) / math.factorial(k) 

    # Chi-squared calculation 
    chi_sq = sum(((count - expected_count)**2) / expected_count for count in observed.values())
    degrees_of_freedom = len(observed) - 1

    p_value = gammaincc(degrees_of_freedom / 2, chi_sq / 2)
    return chi_sq, p_value >= 0.01, p_value # Returns statistic, result and p-value

def test_binary_matrix_rank(random_numbers_C, matrix_size=32):
    """Binary Matrix Rank Test:
       - Generates and tests multiple matrices (more robust).
       - Handles cases where the matrix size doesn't divide evenly.
    """
    n = len(random_numbers_C)
    num_matrices = n // matrix_size  # Number of full matrices we can create
    ranks = []
    for i in range(num_matrices):
        start = i * matrix_size
        end = start + matrix_size
        matrix = [int(x >= 0.5) for x in random_numbers_C[start:end]]
        rank = len(set(tuple(matrix[i:i+matrix_size]) for i in range(0, matrix_size, matrix_size)))
        ranks.append(rank)
    # Analyze the distribution of ranks. 
    # (You'd need to compare the rank distribution to the expected distribution 
    #  for random matrices, which is more involved)
    return ranks, True, 1.0 # Placeholder: needs rank distribution comparison


def statistical_tests(random_numbers_C):
    # Existing Diehard tests...
    bit_string = ''.join(['1' if x >= 0.5 else '0' for x in random_numbers_C])
    p_value, passed = next_bit_test(bit_string)

    print("Additional Statistical Tests:")
    print(f"Universal Statistical Test: {'Passed' if test_universal_statistical(random_numbers_C) else 'Failed'}")
    print(f"Covariance Test: {'Passed' if test_covariance(random_numbers_C) else 'Failed'}")
    print(f"Birthday Spacings Test: {'Passed' if test_birthday_spacings(random_numbers_C) else 'Failed'}")
    print(f"Overlapping Permutations Test: {'Passed' if test_overlapping_permutations(random_numbers_C) else 'Failed'}")
    print(f"Binary Matrix Rank Test: {'Passed' if test_binary_matrix_rank(random_numbers_C) else 'Failed'}")
    print(f"Next-Bit Test: p-value = {p_value:.4f}, Passed = {passed}")

statistical_tests(random_numbers_C)


Additional Statistical Tests:
Universal Statistical Test: Passed
Covariance Test: Passed
Birthday Spacings Test: Passed
Overlapping Permutations Test: Passed
Binary Matrix Rank Test: Passed
Next-Bit Test: p-value = 0.3512, Passed = True


# 1.1e Interpretation
Our first random number generator is a LCG (Linear Congruential Generator), which is a fast method that require minimal memory (one module-m number, often 32 or 64 bits) to retain state. 
Our second random number generator is a CSPRNG (cryptographically secure pseudorandom number generator). While it's technically not as fast as a LCG and requires more memory, it rarely matters in practical situations. As project required, while generating 100000 random numbers, they both take 1e-07s and around 120 MB.
CSPRNG's strength lies in withstanding state compromise extension attacks. 


In [25]:
# 1.2 Monte Carlo Simulation

# generating the shape
def is_inside_shape(x, y):
    return x**2 - abs(x)*y + y**2 <= 25

# the area estimation formula definition
def monte_carlo_area(n, random_generator=random.random):
    inside_count = 0
    for _ in range(n):
        x = random_generator() * 10 - 5  # Scale to -5 to 5
        y = random_generator() * 10 - 5  # Scale to -5 to 5
        if is_inside_shape(x, y):
            inside_count += 1
    return (inside_count / n) * 100  # Area estimate

In [26]:
# Halton Sequence Generator
def halton_sequence(i, base=2):
    f = 1
    r = 0
    while i > 0:
        f /= base
        r += f * (i % base)
        i //= base
    return r

In [79]:
# Estimate area using different methods and sample sizes
for n in [1000, 10000, 100000]:
    # Using LCG
    area_lcg = monte_carlo_area(n, lcg.random)
    # Using CSPRNG
    area_csprng = monte_carlo_area(n, csprng_random)
    # Using random.random()
    area_python_random = monte_carlo_area(n)

    # Using Halton Sequence
    area_halton = monte_carlo_area(n, lambda: halton_sequence(random.randint(1, n)))

    print(f"\nArea Estimation with n = {n}:")
    print(f"LCG: {area_lcg:.2f}")
    print(f"CSPRNG: {area_csprng:.2f}") 
    print(f"Python's random: {area_python_random:.2f}")
    print(f"Halton Sequence: {area_halton:.2f}")


Area Estimation with n = 1000:
LCG: 78.90
CSPRNG: 80.80
Python's random: 78.70
Halton Sequence: 80.10

Area Estimation with n = 10000:
LCG: 79.93
CSPRNG: 79.63
Python's random: 80.94
Halton Sequence: 80.21

Area Estimation with n = 100000:
LCG: 80.44
CSPRNG: 80.47
Python's random: 80.11
Halton Sequence: 80.34
